In [ ]:
import torch
import matplotlib

prefix = "../experiments_test_02_09_2025/"


metric_name_to_model_name_to_metric_mean = {
    "quantile_wasserstein2":{},
    "quantile_hausdorff_distance":{},
    "quantile_l2_distance":{},
    "inverse_quantile_wasserstein2":{},
    "inverse_quantile_hausdorff_distance":{},
    "inverse_quantile_l2_distance":{}
}
metric_name_to_model_name_to_metric_std = {
    "quantile_wasserstein2":{},
    "quantile_hausdorff_distance":{},
    "quantile_l2_distance":{},
    "inverse_quantile_wasserstein2":{},
    "inverse_quantile_hausdorff_distance":{},
    "inverse_quantile_l2_distance":{}
}
(
    model_name_to_idx,
    idx_to_model_name,
    model_name_to_color
) = {}, {}, {}

model_name_to_metric_path = {
    "CPFlow":"convex_potential_flow/metrics.pth",
    # "EntropicOTQuantileRegression/u/FFNN":"entropic_optimal_transport_quantile_regression/u/FFNN/metrics.pth",
    # "EntropicOTQuantileRegression/u/PICNN":"entropic_optimal_transport_quantile_regression/u/PICNN/metrics.pth",
    # # "EntropicOTQuantileRegression/u/PISCNN":"entropic_optimal_transport_quantile_regression/u/PISCNN/metrics.pth",
    # # "EntropicOTQuantileRegression/u/SCFFNN":"entropic_optimal_transport_quantile_regression/u/SCFFNN/metrics.pth",
    # "EntropicOTQuantileRegression/y/FFNN":"entropic_optimal_transport_quantile_regression/y/FFNN/metrics.pth",
    # "EntropicOTQuantileRegression/y/PICNN":"entropic_optimal_transport_quantile_regression/y/PICNN/metrics.pth",
    # # "EntropicOTQuantileRegression/y/PISCNN":"entropic_optimal_transport_quantile_regression/y/PISCNN/metrics.pth",
    # # "EntropicOTQuantileRegression/y/SCFFNN":"entropic_optimal_transport_quantile_regression/y/SCFFNN/metrics.pth",
    # "UnconstrainedAmortizedOTQuantileRegression/u/PISCNN":"unconstrained_amortized_optimal_transport_quantile_regression/u/PISCNN/metrics.pth",
    # "UnconstrainedAmortizedOTQuantileRegression/u/SCFFNN":"unconstrained_amortized_optimal_transport_quantile_regression/u/SCFFNN/metrics.pth",
    # "UnconstrainedAmortizedOTQuantileRegression/y/PISCNN":"unconstrained_amortized_optimal_transport_quantile_regression/y/PISCNN/metrics.pth",
    # "UnconstrainedAmortizedOTQuantileRegression/y/SCFFNN":"unconstrained_amortized_optimal_transport_quantile_regression/y/SCFFNN/metrics.pth",
    # "UnconstrainedOTQuantileRegression/u/PISCNN":"unconstrained_optimal_transport_quantile_regression/u/PISCNN/metrics.pth",
    # "UnconstrainedOTQuantileRegression/u/SCFFNN":"unconstrained_optimal_transport_quantile_regression/u/SCFFNN/metrics.pth",
    # "UnconstrainedOTQuantileRegression/y/PISCNN":"unconstrained_optimal_transport_quantile_regression/y/PISCNN/metrics.pth",
    # "UnconstrainedOTQuantileRegression/y/SCFFNN":"unconstrained_optimal_transport_quantile_regression/y/SCFFNN/metrics.pth",
}

for idx, (model_name, path_to_metric) in enumerate(model_name_to_metric_path.items()):
    model_name_to_idx[model_name] = idx
    idx_to_model_name[idx] = model_name
    model_name_to_color = matplotlib.colormaps['hsv'](idx / len(model_name_to_metric_path))
    metrics_dict = torch.load(f"{prefix}/{path_to_metric}", torch.device("cpu"), weights_only=False)
    
    if 'wasserstein2' in metrics_dict['quantile']:
        quantile_wasserstein2_metric = metrics_dict['quantile']['wasserstein2']
        number_of_XY_samples = quantile_wasserstein2_metric.shape[0]
        quantile_wasserstein2_mean = quantile_wasserstein2_metric.mean()
        quantile_wasserstein2_std = torch.sqrt(quantile_wasserstein2_metric.cov().sum()) / number_of_XY_samples
        metric_name_to_model_name_to_metric_std['quantile_wasserstein2'][model_name] = quantile_wasserstein2_std.item()
        metric_name_to_model_name_to_metric_mean['quantile_wasserstein2'][model_name] = quantile_wasserstein2_mean.item()

    if 'wasserstein2' in metrics_dict['inverse_quantile']:
        inverse_quantile_wasserstein2_metric = metrics_dict['inverse_quantile']['wasserstein2']
        number_of_XY_samples = inverse_quantile_wasserstein2_metric.shape[0]
        inverse_quantile_wasserstein2_mean = inverse_quantile_wasserstein2_metric.mean()
        inverse_quantile_wasserstein2_std = torch.sqrt(inverse_quantile_wasserstein2_metric.cov().sum()) / number_of_XY_samples
        metric_name_to_model_name_to_metric_std['inverse_quantile_wasserstein2'][model_name] = inverse_quantile_wasserstein2_std.item()
        metric_name_to_model_name_to_metric_mean['inverse_quantile_wasserstein2'][model_name] = inverse_quantile_wasserstein2_mean.item()

    if 'hausdorff_distance' in metrics_dict['quantile']:
        quantile_hausdorff_metric = metrics_dict['quantile']['hausdorff_distance']
        number_of_XY_samples = quantile_hausdorff_metric.shape[0]

        quantile_hausdorff_mean = quantile_hausdorff_metric.mean()
        quantile_hausdorff_std = torch.sqrt(quantile_hausdorff_metric.cov().sum()) / number_of_XY_samples

        metric_name_to_model_name_to_metric_std['quantile_hausdorff_distance'][model_name] = quantile_hausdorff_std.item()
        metric_name_to_model_name_to_metric_mean['quantile_hausdorff_distance'][model_name] = quantile_hausdorff_mean.item()

    if 'hausdorff_distance' in metrics_dict['inverse_quantile']:
        inverse_quantile_hausdorff_metric = metrics_dict['inverse_quantile']['hausdorff_distance']
        number_of_XY_samples = inverse_quantile_hausdorff_metric.shape[0]

        inverse_quantile_hausdorff_mean = inverse_quantile_hausdorff_metric.mean()
        inverse_quantile_hausdorff_std = torch.sqrt(inverse_quantile_hausdorff_metric.cov().sum()) / number_of_XY_samples

        metric_name_to_model_name_to_metric_mean['inverse_quantile_hausdorff_distance'][model_name] = inverse_quantile_hausdorff_mean.item()
        metric_name_to_model_name_to_metric_std['inverse_quantile_hausdorff_distance'][model_name] = inverse_quantile_hausdorff_std.item()

    if 'l2_distance' in metrics_dict['quantile']:
        quantile_l2 = metrics_dict['quantile']['l2_distance']
        number_of_XY_samples = quantile_l2.shape[0]
        quantile_l2_mean = quantile_l2.mean()
        quantile_l2_std = torch.sqrt(quantile_l2.cov().sum()) / number_of_XY_samples
        metric_name_to_model_name_to_metric_std['quantile_l2_distance'][model_name] = quantile_l2_std.item()
        metric_name_to_model_name_to_metric_mean['quantile_l2_distance'][model_name] = quantile_l2_mean.item()
    
    if 'l2_distance' in metrics_dict['inverse_quantile']:
        inverse_quantile_l2 = metrics_dict['inverse_quantile']['l2_distance']
        number_of_XY_samples = inverse_quantile_l2.shape[0]
        inverse_quantile_l2_mean = inverse_quantile_l2.mean()
        inverse_quantile_l2_std = torch.sqrt(inverse_quantile_l2.cov().sum()) / number_of_XY_samples
        metric_name_to_model_name_to_metric_std['inverse_quantile_l2_distance'][model_name] = inverse_quantile_l2_std.item()
        metric_name_to_model_name_to_metric_mean['inverse_quantile_l2_distance'][model_name] = inverse_quantile_l2_mean.item()


In [4]:
import torch

metrics = torch.load("../../experiments_test_02_09_2025/fnlvqr_banana/metrics.pth")

for key in metrics['inverse_quantile'].keys():
    metric = torch.stack(metrics['inverse_quantile'][key])
    print(key, metric.std(), metric.mean())

wasserstein2 tensor(0.0106) tensor(0.1547)
unexplained_variance_percentage tensor(2.9666) tensor(49.6721)
monotonicity_violations tensor(0.3143) tensor(7.4631)
sliced_wasserstein2 tensor(0.0062) tensor(0.1508)
kde_kl_divergence tensor(0.0010) tensor(0.0025)
kde_l1_divergence tensor(0.0007) tensor(0.0126)


In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

fig.suptitle("Quantile function approximation")

ax1.set_title("Average L2 distance")
ax1.set_xlabel("Model's index")
ax1.set_ylabel("Average L2 distance")

ax2.set_title("Wasserstein-2")
ax2.set_xlabel("Model's index")
ax2.set_ylabel("Wasserstein-2")

ax3.set_title("Hausdorff Error")
ax3.set_xlabel("Model's index")
ax3.set_ylabel("Hausdorff Error")

for model_name, idx in model_name_to_idx.items():
    if model_name in metric_name_to_model_name_to_metric_mean["quantile_l2_distance"]:
        l2_distance = metric_name_to_model_name_to_metric_mean["quantile_l2_distance"][model_name]
        ax1.scatter(idx, l2_distance, marker=f'${idx}$', label=model_name)

    if model_name in metric_name_to_model_name_to_metric_mean["quantile_wasserstein2"]:
        wasserstein2 = metric_name_to_model_name_to_metric_mean["quantile_wasserstein2"][model_name]
        ax2.scatter(idx, wasserstein2, marker=f'${idx}$', label=model_name)

    
    if model_name in metric_name_to_model_name_to_metric_mean["quantile_hausdorff_distance"]:
        quantile_error = metric_name_to_model_name_to_metric_mean["quantile_hausdorff_distance"][model_name]
        ax3.scatter(idx, quantile_error, marker=f'${idx}$', label=model_name)


ax3.legend(markerscale=2, bbox_to_anchor=[1.01, 1.01], loc='upper left')
plt.tight_layout()
plt.show()


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

fig.suptitle("Inverse quantile function approximation")

ax1.set_title("Average L2 distance")
ax1.set_xlabel("Model's index")
ax1.set_ylabel("Average L2 distance")

ax2.set_title("Wasserstein-2")
ax2.set_xlabel("Model's index")
ax2.set_ylabel("Wasserstein-2")

ax3.set_title("Hausdorff Error")
ax3.set_xlabel("Model's index")
ax3.set_ylabel("Hausdorff Error")

for model_name, idx in model_name_to_idx.items():
    if model_name in metric_name_to_model_name_to_metric_mean["inverse_quantile_l2_distance"]:
        inverse_l2_distance = metric_name_to_model_name_to_metric_mean["inverse_quantile_l2_distance"][model_name]
        ax1.scatter(idx, inverse_l2_distance, marker=f'${idx}$', label=model_name)

    if model_name in metric_name_to_model_name_to_metric_mean["inverse_quantile_wasserstein2"]:
        inverse_wasserstein2 = metric_name_to_model_name_to_metric_mean["inverse_quantile_wasserstein2"][model_name]
        ax2.scatter(idx, inverse_wasserstein2, marker=f'${idx}$', label=model_name)

    
    if model_name in metric_name_to_model_name_to_metric_mean["inverse_quantile_hausdorff_distance"]:
        inverse_quantile_error = metric_name_to_model_name_to_metric_mean["inverse_quantile_hausdorff_distance"][model_name]
        ax3.scatter(idx, inverse_quantile_error, marker=f'${idx}$', label=model_name)

ax3.legend(markerscale=2, bbox_to_anchor=[1.01, 1.01], loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_top5_models(metric_prefix: str, title: str):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(title)

    axes = {
        "l2_distance": ax1,
        "wasserstein2": ax2,
        "hausdorff_distance": ax3,
    }

    # Titles & labels
    ax1.set_title("Average L2 distance")
    ax1.set_xlabel("Model's index")
    ax1.set_ylabel("Average L2 distance")

    ax2.set_title("Wasserstein-2")
    ax2.set_xlabel("Model's index")
    ax2.set_ylabel("Wasserstein-2")

    ax3.set_title("Hausdorff Error")
    ax3.set_xlabel("Model's index")
    ax3.set_ylabel("Hausdorff Error")

    # Collect top 5 models across all metrics (union)
    top_models = set()
    for metric_suffix in axes:
        metric_name = f"{metric_prefix}_{metric_suffix}"
        means = metric_name_to_model_name_to_metric_mean[metric_name]
        sorted_models = sorted(means.items(), key=lambda x: x[1])[:5]
        top_models.update(m for m, _ in sorted_models)

    top_models = list(top_models)

    # Assign consistent colors/markers
    colors = plt.cm.tab10(np.linspace(0, 1, len(top_models)))
    model_to_style = {m: colors[i] for i, m in enumerate(top_models)}

    handles = []

    for metric_suffix, ax in axes.items():
        metric_name = f"{metric_prefix}_{metric_suffix}"
        means = metric_name_to_model_name_to_metric_mean[metric_name]
        stds = metric_name_to_model_name_to_metric_std[metric_name]

        # Plot only top models for this metric
        for model_name in top_models:
            if model_name not in means:
                continue

            idx = model_name_to_idx[model_name]
            mean_val = means[model_name]
            std_val = stds[model_name]

            h = ax.errorbar(
                idx,
                mean_val,
                yerr=std_val,
                fmt="o",
                capsize=5,
                color=model_to_style[model_name],
                label=model_name,
            )
            # Collect one handle per model (for shared legend)
            if metric_suffix == "hausdorff_distance":
                handles.append(h)

    # Single shared legend next to last subplot
    ax3.legend(
        handles,
        [h.get_label() for h in handles],
        markerscale=2,
        bbox_to_anchor=(1.01, 1.01),
        loc="upper left"
    )

    plt.tight_layout()
    plt.show()


# Plot both sets
plot_top5_models("quantile", "Quantile function approximation (Top-5)")
plot_top5_models("inverse_quantile", "Inverse quantile function approximation (Top-5)")

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

fig.suptitle("Quantile function approximation")

ax1.set_title("Average L2 distance")
ax1.set_xlabel("Model's index")
ax1.set_ylabel("Average L2 distance")

ax2.set_title("Wasserstein-2")
ax2.set_xlabel("Model's index")
ax2.set_ylabel("Wasserstein-2")

ax3.set_title("Hausdorff Error")
ax3.set_xlabel("Model's index")
ax3.set_ylabel("Hausdorff Error")

for model_name, idx in model_name_to_idx.items():
    if model_name in metric_name_to_model_name_to_metric_mean["quantile_l2_distance"]:
        l2_distance = metric_name_to_model_name_to_metric_mean["quantile_l2_distance"][model_name]
        ax1.scatter(idx, l2_distance, marker=f'${idx}$', label=model_name)

    if model_name in metric_name_to_model_name_to_metric_mean["quantile_wasserstein2"]:
        wasserstein2 = metric_name_to_model_name_to_metric_mean["quantile_wasserstein2"][model_name]
        ax2.scatter(idx, wasserstein2, marker=f'${idx}$', label=model_name)

    
    if model_name in metric_name_to_model_name_to_metric_mean["quantile_hausdorff_distance"]:
        quantile_error = metric_name_to_model_name_to_metric_mean["quantile_hausdorff_distance"][model_name]
        ax3.scatter(idx, quantile_error, marker=f'${idx}$', label=model_name)


ax3.legend(markerscale=2, bbox_to_anchor=[1.01, 1.01], loc='upper left')
plt.tight_layout()
plt.show()


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

fig.suptitle("Inverse quantile function approximation")

ax1.set_title("Average L2 distance")
ax1.set_xlabel("Model's index")
ax1.set_ylabel("Average L2 distance")

ax2.set_title("Wasserstein-2")
ax2.set_xlabel("Model's index")
ax2.set_ylabel("Wasserstein-2")

ax3.set_title("Hausdorff Error")
ax3.set_xlabel("Model's index")
ax3.set_ylabel("Hausdorff Error")

for model_name, idx in model_name_to_idx.items():
    if model_name in metric_name_to_model_name_to_metric_mean["inverse_quantile_l2_distance"]:
        l2_distance = metric_name_to_model_name_to_metric_mean["inverse_quantile_l2_distance"][model_name]
        ax1.scatter(idx, l2_distance, marker=f'${idx}$', label=model_name)

    if model_name in metric_name_to_model_name_to_metric_mean["inverse_quantile_wasserstein2"]:
        wasserstein2 = metric_name_to_model_name_to_metric_mean["inverse_quantile_wasserstein2"][model_name]
        ax2.scatter(idx, wasserstein2, marker=f'${idx}$', label=model_name)

    
    if model_name in metric_name_to_model_name_to_metric_mean["inverse_quantile_hausdorff_distance"]:
        quantile_error = metric_name_to_model_name_to_metric_mean["inverse_quantile_hausdorff_distance"][model_name]
        ax3.scatter(idx, quantile_error, marker=f'${idx}$', label=model_name)


ax3.legend(markerscale=2, bbox_to_anchor=[1.01, 1.01], loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))
fig.suptitle("Performance Comparison for Top 5 Models", fontsize=16)

index_to_metric = {}

all_indices = []
all_wasserstein2, all_wasserstein2_stds = [], []
all_quantile_error, all_quantile_error_stds = [], []
all_l2_distance, all_l2_distance_stds = [], []

for model_name, idx in model_name_to_idx.items():
    l2_distance_mean = metric_name_to_model_name_to_metric_mean["l2_distance"][model_name]
    wasserstein2_mean = metric_name_to_model_name_to_metric_mean["wasserstein2"][model_name]
    quantile_error_mean = metric_name_to_model_name_to_metric_mean["quantile_error"][model_name]

    l2_distance_std = metric_name_to_model_name_to_metric_std["l2_distance"][model_name]
    wasserstein2_std = metric_name_to_model_name_to_metric_std["wasserstein2"][model_name]
    quantile_error_std = metric_name_to_model_name_to_metric_std["quantile_error"][model_name]

    all_wasserstein2.append(wasserstein2_mean)
    all_quantile_error.append(quantile_error_mean)
    all_l2_distance.append(l2_distance_mean)

    all_wasserstein2_stds.append(wasserstein2_std)
    all_quantile_error_stds.append(quantile_error_std)
    all_l2_distance_stds.append(l2_distance_std)

    all_indices.append(idx)

all_wasserstein2_tensor = torch.tensor(all_wasserstein2)
all_quantile_error_tensor = torch.tensor(all_quantile_error)
all_l2_distance_tensor = torch.tensor(all_l2_distance)

top_5_l2_distance_models = all_l2_distance_tensor.argsort()[:5]
top_5_wasserstein2_models = all_wasserstein2_tensor.argsort()[:5]
top_5_quantile_error_models = all_quantile_error_tensor.argsort()[:5]

def plot_barchart(axis, title, metric_name, top_indices, model_name_to_metric_mean, model_name_to_metric_std, lower_is_better=True):
    axis.set_title(title)

    model_names = [idx_to_model_name[i.item()] for i in top_indices]
    means = [model_name_to_metric_mean[model_name] for model_name in model_names]
    stds = [model_name_to_metric_std[model_name] for model_name in model_names]

    sorted_data = sorted(zip(means, stds, model_names), reverse=not lower_is_better)
    sorted_means, sorted_stds, sorted_names = zip(*sorted_data)

    y_pos = np.arange(len(sorted_names))
    colors = plt.cm.viridis(np.linspace(0, 1, len(y_pos)))

    axis.barh(y_pos, sorted_means, xerr=sorted_stds, align='center', color=colors, capsize=5)
    axis.set_yticks(y_pos)
    axis.set_yticklabels(sorted_names)
    axis.invert_yaxis()
    axis.set_xlabel(metric_name)
    axis.grid(True, which="major", axis='x', ls="--", c='0.8')

plot_barchart(
    axis=ax1,
    title="Mean L2 distance",
    metric_name="Mean L2 distance",
    top_indices=top_5_l2_distance_models,
    model_name_to_metric_mean=metric_name_to_model_name_to_metric_mean["l2_distance"],
    model_name_to_metric_std=metric_name_to_model_name_to_metric_std["l2_distance"],
    lower_is_better=True
)

plot_barchart(
    axis=ax2,
    title="Wasserstein-2",
    metric_name="$W_2$ Distance",
    top_indices=top_5_wasserstein2_models,
    model_name_to_metric_mean=metric_name_to_model_name_to_metric_mean["wasserstein2"],
    model_name_to_metric_std=metric_name_to_model_name_to_metric_std["wasserstein2"],
    lower_is_better=True
)

plot_barchart(
    axis=ax3,
    title="Quantile Error",
    metric_name="Quantile Error",
    top_indices=top_5_quantile_error_models,
    model_name_to_metric_mean=metric_name_to_model_name_to_metric_mean["quantile_error"],
    model_name_to_metric_std=metric_name_to_model_name_to_metric_std["quantile_error"],
    lower_is_better=True
)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

models_names_from_epsilon_experiment = [model_name for model_name in model_name_to_idx.keys() if "Epsilon" in model_name]

epsilons = []
all_l2_distance_means, all_l2_distance_stds = [], []
all_wasserstein2_means, all_wasserstein2_stds = [], []
all_quantile_error_means, all_quantile_error_stds = [], []

for model_name in models_names_from_epsilon_experiment:
    try:
        epsilon_str = model_name.split('/')[-1].replace('_', '-')
        epsilons.append(float(epsilon_str))
    except (ValueError, IndexError):
        print(f"Warning: Could not parse epsilon from path: {model_name}")
        continue

    l2_distance_mean = metric_name_to_model_name_to_metric_mean["l2_distance"][model_name]
    wasserstein2_mean = metric_name_to_model_name_to_metric_mean["wasserstein2"][model_name]
    quantile_error_mean = metric_name_to_model_name_to_metric_mean["quantile_error"][model_name]

    l2_distance_std = metric_name_to_model_name_to_metric_std["l2_distance"][model_name]
    wasserstein2_std = metric_name_to_model_name_to_metric_std["wasserstein2"][model_name]
    quantile_error_std = metric_name_to_model_name_to_metric_std["quantile_error"][model_name]

    all_l2_distance_means.append(l2_distance_mean)
    all_wasserstein2_means.append(wasserstein2_mean)
    all_quantile_error_means.append(quantile_error_mean)

    all_l2_distance_stds.append(l2_distance_std)
    all_wasserstein2_stds.append(wasserstein2_std)
    all_quantile_error_stds.append(quantile_error_std)

if epsilons:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle("Divergences vs. Epsilon for Entropic OT")

    ax1.set_title("L2 distance")
    ax1.set_ylabel("L2 distance")

    ax2.set_title("Wasserstein-2")
    ax2.set_ylabel("Wasserstein-2")

    ax3.set_title("Quantile Error")
    ax3.set_ylabel("Quantile Error")

    sorted_data = sorted(zip(
        epsilons,
        all_l2_distance_means, all_l2_distance_stds,
        all_wasserstein2_means, all_wasserstein2_stds,
        all_quantile_error_means, all_quantile_error_stds
    ))

    (
        sorted_epsilons,
        sorted_l2_distance_means, sorted_l2_distance_stds,
        sorted_wasserstein2_means, sorted_wasserstein2_stds,
        sorted_quantile_error_means, sorted_quantile_error_stds
    ) = zip(*sorted_data)

    common_errorbar_settings = {'fmt': '-o', 'capsize': 5, 'elinewidth': 1.5, 'markersize': 6, 'color': 'royalblue', 'markerfacecolor': 'royalblue'}

    ax1.errorbar(sorted_epsilons, sorted_l2_distance_means, yerr=sorted_l2_distance_stds, **common_errorbar_settings)
    ax2.errorbar(sorted_epsilons, sorted_wasserstein2_means, yerr=sorted_wasserstein2_stds, **common_errorbar_settings)
    ax3.errorbar(sorted_epsilons, sorted_quantile_error_means, yerr=sorted_quantile_error_stds, **common_errorbar_settings)

    for ax in [ax1, ax2, ax3]:
        ax.set_xscale('log')
        ax.set_xlabel("Epsilon")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
else:
    print("No data to plot")

In [ ]:
import matplotlib.pyplot as plt

models_names_from_batch_variations_experiment = [model_name for model_name in model_name_to_idx.keys() if "LatentSampleSize" in model_name]

latent_sample_sizes = []
all_l2_distance_means, all_l2_distance_stds = [], []
all_wasserstein2_means, all_wasserstein2_stds = [], []
all_quantile_error_means, all_quantile_error_stds = [], []

for model_name in models_names_from_batch_variations_experiment:
    try:
        latent_sample_size_str = model_name.split('/')[-1]
        latent_sample_sizes.append(int(latent_sample_size_str))
    except (ValueError, IndexError):
        print(f"Warning: Could not parse latent sample size from path: {model_name}")
        continue


    l2_distance_mean = metric_name_to_model_name_to_metric_mean["l2_distance"][model_name]
    wasserstein2_mean = metric_name_to_model_name_to_metric_mean["wasserstein2"][model_name]
    quantile_error_mean = metric_name_to_model_name_to_metric_mean["quantile_error"][model_name]

    l2_distance_std = metric_name_to_model_name_to_metric_std["l2_distance"][model_name]
    wasserstein2_std = metric_name_to_model_name_to_metric_std["wasserstein2"][model_name]
    quantile_error_std = metric_name_to_model_name_to_metric_std["quantile_error"][model_name]

    all_l2_distance_means.append(l2_distance_mean)
    all_wasserstein2_means.append(wasserstein2_mean)
    all_quantile_error_means.append(quantile_error_mean)

    all_l2_distance_stds.append(l2_distance_std)
    all_wasserstein2_stds.append(wasserstein2_std)
    all_quantile_error_stds.append(quantile_error_std)

if latent_sample_sizes:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle("Divergences vs. Latent Sample Size for psi estimate in Entropic OT")

    ax1.set_title("Mean L2 distance")
    ax1.set_ylabel("Mean L2 distance")

    ax2.set_title("Wasserstein-2")
    ax2.set_ylabel("Wasserstein-2")

    ax3.set_title("Quantile Error")
    ax3.set_ylabel("Quantile Error")

    sorted_data = sorted(zip(
        latent_sample_sizes,
        all_l2_distance_means, all_l2_distance_stds,
        all_wasserstein2_means, all_wasserstein2_stds,
        all_quantile_error_means, all_quantile_error_stds
    ))

    (
        sorted_latent_sample_sizes,
        sorted_l2_distance_means, sorted_l2_distance_stds,
        sorted_wasserstein2_means, sorted_wasserstein2_stds,
        sorted_quantile_error_means, sorted_quantile_error_stds
    ) = zip(*sorted_data)

    common_errorbar_settings = {'fmt': '-o', 'capsize': 5, 'elinewidth': 1.5, 'markersize': 6, 'color': 'royalblue', 'markerfacecolor': 'royalblue'}
    ax1.errorbar(sorted_latent_sample_sizes, sorted_l2_distance_means, yerr=sorted_l2_distance_stds, **common_errorbar_settings)
    ax2.errorbar(sorted_latent_sample_sizes, sorted_wasserstein2_means, yerr=sorted_wasserstein2_stds, **common_errorbar_settings)
    ax3.errorbar(sorted_latent_sample_sizes, sorted_quantile_error_means, yerr=sorted_quantile_error_stds, **common_errorbar_settings)

    for ax in [ax1, ax2, ax3]:
        ax.set_xlabel("Latent Sample Size")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
else:
    print("No data to plot")

In [ ]:
import matplotlib.pyplot as plt

models_names_from_strong_convexity_experiment = [model_name for model_name in model_name_to_idx.keys() if "UnconstrainedOTQuantileRegression" in model_name]

strong_convexity_values = []
all_l2_distance_means, all_l2_distance_stds = [], []
all_wasserstein2_means, all_wasserstein2_stds = [], []
all_quantile_error_means, all_quantile_error_stds = [], []

for model_name in models_names_from_strong_convexity_experiment:
    try:
        strong_convexity_value_str = model_name.split('/')[-1].replace('_', '-')
        strong_convexity_values.append(float(strong_convexity_value_str))
    except (ValueError, IndexError):
        print(f"Warning: Could not parse strong convexity value from path: {model_name}")
        continue

    l2_distance_mean = metric_name_to_model_name_to_metric_mean["l2_distance"][model_name]
    wasserstein2_mean = metric_name_to_model_name_to_metric_mean["wasserstein2"][model_name]
    quantile_error_mean = metric_name_to_model_name_to_metric_mean["quantile_error"][model_name]

    l2_distance_std = metric_name_to_model_name_to_metric_std["l2_distance"][model_name]
    wasserstein2_std = metric_name_to_model_name_to_metric_std["wasserstein2"][model_name]
    quantile_error_std = metric_name_to_model_name_to_metric_std["quantile_error"][model_name]

    all_l2_distance_means.append(l2_distance_mean)
    all_wasserstein2_means.append(wasserstein2_mean)
    all_quantile_error_means.append(quantile_error_mean)

    all_l2_distance_stds.append(l2_distance_std)
    all_wasserstein2_stds.append(wasserstein2_std)
    all_quantile_error_stds.append(quantile_error_std)

if strong_convexity_values:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle("Divergences vs. Strong Convexity for Unregularized OT")

    ax1.set_title("Mean L2 distance")
    ax1.set_ylabel("Mean L2 distance")

    ax2.set_title("Wasserstein-2")
    ax2.set_ylabel("Wasserstein-2")

    ax3.set_title("Quantile Error")
    ax3.set_ylabel("Quantile Error")

    sorted_data = sorted(zip(
        strong_convexity_values,
        all_l2_distance_means, all_l2_distance_stds,
        all_wasserstein2_means, all_wasserstein2_stds,
        all_quantile_error_means, all_quantile_error_stds
    ))

    (
        sorted_strong_convexity_values,
        sorted_l2_distance_means, sorted_l2_distance_stds,
        sorted_wasserstein2_means, sorted_wasserstein2_stds,
        sorted_quantile_error_means, sorted_quantile_error_stds
    ) = zip(*sorted_data)

    common_errorbar_settings = {'fmt': '-o', 'capsize': 5, 'elinewidth': 1.5, 'markersize': 6, 'color': 'royalblue', 'markerfacecolor': 'royalblue'}

    ax1.errorbar(sorted_strong_convexity_values, sorted_l2_distance_means, yerr=sorted_l2_distance_stds, **common_errorbar_settings)
    ax2.errorbar(sorted_strong_convexity_values, sorted_wasserstein2_means, yerr=sorted_wasserstein2_stds, **common_errorbar_settings)
    ax3.errorbar(sorted_strong_convexity_values, sorted_quantile_error_means, yerr=sorted_quantile_error_stds, **common_errorbar_settings)

    for ax in [ax1, ax2, ax3]:
        ax.set_xscale('log')
        ax.set_xlabel("Strong Convexity")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
else:
    print("No data to plot")

In [ ]:
import matplotlib.pyplot as plt
import torch

model_name_to_activation_function_name = {
    "EntropicOTQuantileRegression/CELU":"CELU",
    "EntropicOTQuantileRegression/ELU":"ELU",
    "EntropicOTQuantileRegression/GELU":"GELU",
    "EntropicOTQuantileRegression/Mish":"Mish",
    "EntropicOTQuantileRegression/SiLU":"SiLU",
    "EntropicOTQuantileRegression/Softplus":"Softplus",
    "EntropicOTQuantileRegression/Softsign":"Softsign",
    "EntropicOTQuantileRegression/Tanh":"Tanh",
}

plt.style.use('seaborn-v0_8-whitegrid')
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(22, 7))
fig.suptitle("Performance of Different Non-Linear Activation Functions in Entropic OT", fontsize=16)

colors = plt.cm.viridis(torch.linspace(0, 1, len(model_name_to_activation_function_name)))

ax1.bar(
    model_name_to_activation_function_name.values(),
    [
        metric_name_to_model_name_to_metric_mean["l2_distance"][name] for name in model_name_to_activation_function_name.keys()
    ],
    yerr=[
        metric_name_to_model_name_to_metric_std["l2_distance"][name] for name in model_name_to_activation_function_name.keys()
    ],
    color=colors,
    capsize=5
)
ax1.set_title("Mean L2 distance")
ax1.set_ylabel("Negative Log-Likelihood (Mean)")

ax2.bar(
    model_name_to_activation_function_name.values(),
        [
        metric_name_to_model_name_to_metric_mean["wasserstein2"][name] for name in model_name_to_activation_function_name.keys()
    ],
    yerr=[
        metric_name_to_model_name_to_metric_std["wasserstein2"][name] for name in model_name_to_activation_function_name.keys()
    ],
    color=colors,
    capsize=5
)
ax2.set_title("Wasserstein-2 Distance")
ax2.set_ylabel("W2 Distance (Mean)")
ax2.set_xlabel("Activation Function", fontsize=12)

ax3.bar(
    model_name_to_activation_function_name.values(),
    [
        metric_name_to_model_name_to_metric_mean["quantile_error"][name] for name in model_name_to_activation_function_name.keys()
    ],
    yerr=[
        metric_name_to_model_name_to_metric_std["quantile_error"][name] for name in model_name_to_activation_function_name.keys()
    ],
    color=colors, capsize=5)
ax3.set_title("Quantile Similarity")
ax3.set_ylabel("Similarity (Mean)")

for ax in [ax1, ax2, ax3]:
    ax.tick_params(axis='x', rotation=45)
    ax.set_axisbelow(True)

plt.tight_layout(rect=[0, 0.05, 1, 0.96])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import torch

model_name_to_activation_function_name = {
    "EntropicOTQuantileRegression/GELU":"GELU",
    "EntropicOTQuantileRegression/Softplus":"Softplus",
    "EntropicOTQuantileRegression/PICNN":"PICNN",
}

plt.style.use('seaborn-v0_8-whitegrid')
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(22, 7))
fig.suptitle("Performance of Convex Neural Network vs Feed Forward Neural Network in Entropic OT", fontsize=16)

colors = plt.cm.viridis(torch.linspace(0, 1, len(model_name_to_activation_function_name)))

ax1.bar(
    model_name_to_activation_function_name.values(),
    [
        metric_name_to_model_name_to_metric_mean["l2_distance"][name] for name in model_name_to_activation_function_name.keys()
    ],
    yerr=[
        metric_name_to_model_name_to_metric_std["l2_distance"][name] for name in model_name_to_activation_function_name.keys()
    ],
    color=colors,
    capsize=5
)
ax1.set_title("Mean L2 distance")
ax1.set_ylabel("Negative Log-Likelihood (Mean)")

ax2.bar(
    model_name_to_activation_function_name.values(),
        [
        metric_name_to_model_name_to_metric_mean["wasserstein2"][name] for name in model_name_to_activation_function_name.keys()
    ],
    yerr=[
        metric_name_to_model_name_to_metric_std["wasserstein2"][name] for name in model_name_to_activation_function_name.keys()
    ],
    color=colors,
    capsize=5
)
ax2.set_title("Wasserstein-2 Distance")
ax2.set_ylabel("W2 Distance (Mean)")
ax2.set_xlabel("Activation Function", fontsize=12)

ax3.bar(
    model_name_to_activation_function_name.values(),
    [
        metric_name_to_model_name_to_metric_mean["quantile_error"][name] for name in model_name_to_activation_function_name.keys()
    ],
    yerr=[
        metric_name_to_model_name_to_metric_std["quantile_error"][name] for name in model_name_to_activation_function_name.keys()
    ],
    color=colors, capsize=5)
ax3.set_title("Quantile Similarity")
ax3.set_ylabel("Similarity (Mean)")

for ax in [ax1, ax2, ax3]:
    ax.tick_params(axis='x', rotation=45)
    ax.set_axisbelow(True)

plt.tight_layout(rect=[0, 0.05, 1, 0.96])
plt.show()